# Offshore Wind Farming
## Objective and Prerequisites

In this example, we will solve the problem of how to minimize the cost of laying underwater cables to collect electricity produced by an offshore wind farm. We will construct a mixed-integer programming (MIP)  model of this problem, implement this model in the Gurobi Python interface, and compute an optimal solution.

We will also investigate the use of lazy constraints and where they can help (and where they cannot).

## Motivation

Global climate change has already had observable effects on the environment. Glaciers have shrunk, ice on rivers and lakes is breaking up earlier than expected, plant and animal species have  been affected and trees are flowering sooner than expected. The potential future effects of global climate change include more frequent wildfires, longer periods of drought in some regions and an increase in the number, duration and intensity of tropical storms. [1]

Climate change mitigation consists of actions to limit the magnitude or rate of global warming and its related effects.
The first challenge for climate change mitigation is eliminating the burning of coal, oil and, eventually, natural gas. This is perhaps the most daunting challenge as denizens of richer nations literally eat, wear, work, play and even sleep on the products made from fossil fuels. Also, citizens of developing nations want and arguably deserve the same comforts. There are no perfect solutions for reducing dependence on fossil fuels (for example, carbon neutral biofuels can drive up the price of food and lead to forest destruction, and while nuclear power does not emit greenhouse gases, it does produce radioactive waste). Other alternatives include plant-derived plastics, biodiesel, and wind power. [2]

Offshore wind power is the use of wind farms constructed in bodies of water, usually in the ocean, to harvest wind energy to generate electricity. Higher wind speeds are available offshore compared to on land, so offshore wind power’s electricity generation is higher per amount of capacity installed. 

The advantage of locating wind turbines offshore is that the wind is much stronger off the coasts, and unlike wind over the continent, offshore breezes can be strong in the afternoon, matching the time when people are using the most electricity. Offshore turbines can also be located close to the load centers along the coasts, such as large cities, eliminating the need for new long-distance transmission lines.

## Problem Description

An offshore wind farm is a collection of wind turbines placed at sea to take advantage of the strong offshore winds. These strong winds produce more electricity, but offshore wind farms are more expensive to install and operate than those on land.

We will use a MIP model to reduce part of the cost of building an offshore wind farm. We will compute a plan for how to lay the underwater cables that connect the turbines. These cables are necessary to transfer the power produced by the turbines to land. The plan we compute will minimize the cost to install the underwater cables, while ensuring that each turbine is connected to the shore and each cable has sufficient capacity to handle the electrical current generated.

The goal of this optimization problem is to decide which cables should be laid to connect the wind farm power network at a minimum cost.

The model of offshore wind farming optimization problem is an instance of a more general optimization model known as fixed charge network flow problem. Fixed charge network flow problems can be applied to a large number of business problems -for example, in the planning of communication and transport networks.

## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi Optimizer solves such models using state-of-the-art mathematics and computer science.

## Data input

In [20]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
import networkx as nx
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
from shapely.geometry import LineString

# Read in the data
df = pd.read_excel('offshore.xlsx')
price_per_km = 1e6
capacity = 69.4 # Amount of MW that a cable can carry
produced = 8

# Set index
df.set_index('Name',inplace=True)
names = df.index
turbines = [n for n in names if n != 'Plat']

# Define connections
connections = [(o, d) for o in names for d in names if o != d and o != 'Plat']

# Calculate distances
distances = {(o,d): np.sqrt((df.loc[o, 'Easting'] - df.loc[d, 'Easting'])**2 + 
                     (df.loc[o, 'Northing'] - df.loc[d, 'Northing'])**2) / 1000 # Convert to km
            for (o,d) in connections}

# Check if crossing
def check_if_crossing(c1, c2):
    line1 = LineString([(df.loc[c1[0], 'Easting'], df.loc[c1[0], 'Northing']), 
                        (df.loc[c1[1], 'Easting'], df.loc[c1[1], 'Northing'])])
    line2 = LineString([(df.loc[c2[0], 'Easting'], df.loc[c2[0], 'Northing']), 
                        (df.loc[c2[1], 'Easting'], df.loc[c2[1], 'Northing'])])
    
    if c1[0] in [c2[0], c2[1]] or c1[1] in [c2[0], c2[1]]:
        return False
    return line1.crosses(line2)

## Initial visualization

In [11]:
# Define the positions
pos = {n : (df.loc[n, 'Easting'], df.loc[n, 'Northing']) for n in names}

# Draw the units only ones
H = nx.Graph()
H.add_nodes_from([n for n in names])
nx.draw_networkx_nodes(H, pos, node_size = 100, node_color='y')
nx.draw_networkx_labels(H, pos);
plt.rcParams["figure.figsize"] = (200,100)

## Initial model
The initial model is pretty simple. We have the following variables:
- $x_{i,j}$ denotes whether turbine $i$ is connected to node $j$
- $f_{i,j}$ denotes the flow between turbine $i$ and node $j$

We then have to ensure that:
- all turbines are connected: $\sum_{j} x_{i,j}=1$
- the flow balance holds: $\sum_j f_{i,j} = \sum_k f_{k,i} + P$ where $P$ is the power produced at a turbine
- Connect the flow and the connection choice: $f_{i,j} \leq Cx_{i,j}$, where $C$ is the capacity of the cable.

The objective function is to minimize the installation cost, which scales linearly with the distances between the nodes: $\sum_{i,j} d_{i,j}x_{i,j}$

In [18]:
model = gp.Model()

x = model.addVars(connections, vtype=gp.GRB.BINARY, name='install')
f = model.addVars(connections, name='flow')

model.addConstrs((x.sum(i,'*') == 1 for i in turbines), name='All connected')
model.addConstrs((f.sum(i,'*') == f.sum('*',i) + produced for i in turbines), 
                 name='Flow')
model.addConstrs((f[c] <= capacity*x[c] for c in connections), 'Link f and x')

model.setObjective(x.prod(distances))

model.optimize()

## Strengthening the formulation

However, this performance is not good. Why? The problem is that the flows are all multiples of $P$, however the capacity of the cable is not. Therefore there is an overlap, a part of the cable that cannot be used.

The solution? Round the capacity of the cable down to be a multiple of the power produced, i.e.:

$$\bar{C} = \lfloor \frac{C}{P}\rfloor P$$

This way we get:

In [25]:
import math

c_bar = math.floor(capacity/produced)*produced

model = gp.Model()

x = model.addVars(connections, vtype=gp.GRB.BINARY, name='install')
f = model.addVars(connections, name='flow')

model.addConstrs((x.sum(i,'*') == 1 for i in turbines), name='All connected')
model.addConstrs((f.sum(i,'*') == f.sum('*',i) + produced for i in turbines), 
                 name='Flow')
model.addConstrs((f[c] <= c_bar*x[c] for c in connections), 'Link f and x')

model.setObjective(x.prod(distances))

model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 960 rows, 1800 columns and 4470 nonzeros
Model fingerprint: 0x5d0fa7e7
Variable types: 900 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 30 rows and 30 columns
Presolve time: 0.01s
Presolved: 930 rows, 1770 columns, 4410 nonzeros
Variable types: 900 continuous, 870 integer (870 binary)

Root relaxation: objective 3.989381e+01, 1216 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   39.89381    0   28          -   39.89381      -     -    0s
H    0     0                      65.5212480   39.89381  39.1%     -  

## Introducing non-crossing constraints

So far we assumed that the cables can potentially cross, however this is not a realistic requirement: if the lower cable was to fail, one would have to remove the upper one to access is, which is hugely impractical. Therefore, we want to exclude this.

To do so, we simply have to make sure that no two crossing connections $(i,j)$ and $(i',j')$ are built at the same time, i.e.:

$$x_{i,j} + x_{i',j'} \leq 1$$

The problem is that in order to add this constraint to the model, we need to precompute all potentially crossing cables. Let's try:

In [26]:
for i1, c1 in enumerate(connections):
    for i2, c2 in enumerate(connections):
        if i2 > i1 and check_if_crossing(c1,c2):
            model.addConstr(x[c1] + x[c2] <= 1)
            
model.reset()
model.optimize()

Discarded solution information
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 77048 rows, 1800 columns and 156646 nonzeros
Model fingerprint: 0x59d4ec25
Variable types: 900 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 62063 rows and 16 columns
Presolve time: 2.19s
Presolved: 14985 rows, 1784 columns, 136110 nonzeros
Variable types: 900 continuous, 884 integer (884 binary)
Found heuristic solution: objective 128.6438835
Extra simplex iterations after uncrush: 93

Root relaxation: objective 3.989381e+01, 592 iterations, 0.17 seconds (0.18 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 

## Introducing lazy constraints

However, this is quite slow. Can't we do better? Wouldn't it be nice to only add the crossing constraints which are actually violated? Well, callbacks allow you to do just that. With callbacks, you can interact with Gurobi during the solve process. So let's try that:

> Note: lazy constraints are not a magic bullet, but are only to be used when you have a very large number of constraints to generate, where only a few are actually going to be relevant for a (good) solution to the problem.

In [29]:
model = gp.Model()

x = model.addVars(connections, vtype=gp.GRB.BINARY, name='install')
f = model.addVars(connections, name='flow')

model.addConstrs((x.sum(i,'*') == 1 for i in turbines), name='All connected')
model.addConstrs((f.sum(i,'*') == f.sum('*',i) + produced for i in turbines), 
                 name='Flow')
model.addConstrs((f[c] <= c_bar*x[c] for c in connections), 'Link f and x')

model.setObjective(x.prod(distances))

def non_crossing_callback(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._variables)
        selected = [c for c in model._variables if vals[c] > 0.5]
        
        # If they are crossing, add lazy constraint
        for c1 in selected:
            for c2 in selected:
                if c1 != c2 and check_if_crossing(c1, c2):
                    model.cbLazy(model._variables[c1] + model._variables[c2] <= 1)
                    
model.Params.lazyConstraints = 1
model._variables = x
model.optimize(non_crossing_callback)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 960 rows, 1800 columns and 4470 nonzeros
Model fingerprint: 0x5d0fa7e7
Variable types: 900 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [6e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 30 rows and 30 columns
Presolve time: 0.02s
Presolved: 930 rows, 1770 columns, 4410 nonzeros
Variable types: 900 continuous, 870 integer (870 binary)

Root relaxation: objective 3.989381e+01, 1216 iterations, 0.03 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   39.89381    0   28          -   39.89381      -     -    0s
H    0     0                 

## Drawing the final solution

In [30]:
# Draw the edges that actually exist
G = nx.Graph()
G.add_edges_from([(i, j) for (i,j) in connections if x[i,j].x > 0.5])
nx.draw_networkx(G, pos, node_size = 100, node_color='y')
nx.draw_networkx_labels(H, pos);
plt.rcParams["figure.figsize"] = (200,100)